# AI Domo Agent

This notebook tests that we can run `strategic overview` similar to Daniel's AI agent


In [64]:
import sys
!{sys.executable} -m pip install --upgrade reportlab -q


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: pip install --upgrade pip


In [2]:
import requests
import json
import os
from pathlib import Path
from dotenv import load_dotenv
import pandas as pd
from pydomo import Domo
import duckdb
import warnings
warnings.filterwarnings('ignore')

env_path = Path.cwd() / ".env"
if env_path.exists():
    load_dotenv(env_path)


/var/folders/v4/_m77btxd2kbglnwsqvbd13dm0000gn/T/ipykernel_97630/729352324.py:6: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [28]:
pd.set_option('display.max_columns', None)
# Access your variables
DOMO_CLIENT_ID = os.getenv('DOMO_CLIENT_ID')
DOMO_SECRET_KEY = os.getenv('DOMO_SECRET_KEY')
OPEN_ROUTER_KEY = os.getenv('OPEN_ROUTER_KEY')

# Initialize Domo client
domo = Domo(DOMO_CLIENT_ID, DOMO_SECRET_KEY, api_host='api.domo.com')

## 1. Strategic Overview

Query 1: Regional Performance Context (Agent View of Operations | Property KPI Scorecard | PROD)
Business Objective: Provide portfolio context for the scope selected by the user
* Apply filters; Current Property Filter, Regional scope Filter, Stabilized Filter, and Lease-Up Exclusion
* Count ALL remaining properties in filtered scope using record_look_up_code
* Calculate data completeness breakdown for all properties in filtered scope
* Calculate regional averages for Property Scores and category scores across all properties in filtered scope
* This query provides the "Properties Analyzed" number for the report metadata

In [4]:
selected_dataset = '123783d1-459b-41c8-87ba-6468c8f7edaf' # pdb
selected_dataset = '90339811-aa5c-4e35-835c-714f161ba93e' # kpi
selected_dataset = '62955074-e53f-4edf-a009-365da7564b0f' # kpi agent view
print('selected_dataset', selected_dataset)

df = domo.ds_get(selected_dataset)
print(df.shape)
df.head(1) # Display first few rows of the dataframe

selected_dataset 62955074-e53f-4edf-a009-365da7564b0f
(2983, 77)


,record_record_id_,meas_mo,record_property_name,record_corporate_operating_office,record_asset_class,pms_linked,record_property_sub_type,record_accounting_system_of_record,record_takeover_date,record_state,record_ownership_group_name,record_svp_assignment,record_latitude,record_longitude,record_look_up_code,division_president,prop_type_new,record_vice_president_assignment,record_regional_assignment,record_regional_vp_assignment,record_rltd___rmd_assignment,Market Performance Score,Site Value & Risk Mgmt Score,Investment Results Score,Community Standards Score,Customer Experience,const,Property Score,kpi_gpr actual,kpi_gpr budget,kpi_completed wo within 48 hrs,kpi_total wo,kpi_noi actual,kpi_noi budget,kpi_collections,kpi_charges,kpi_delinquency,kpi_ap aging 60+ days,kpi_total ap,kpi_budget total,kpi_occupied,kpi_units,kpi_npr actual,kpi_npr budget,kpi_vend_pref_actl,kpi_vend_pref_elig,kpi_gpr variance,kpi_gpr variance %,kpi_npr variance,kpi_npr variance %,yoy_same_store,kpi_noi variance,kpi_noi variance %,kpi_wo completion %,kpi_collection %,kpi_ap_aging %,kpi_vendor_compliance %,kpi_risk_training_score,kpi_occupancy %,Exceptions Source,Exceptions Detail,Missing Budget,kpi_gpr score,kpi_ap aging score,kpi_collection score,kpi_lockton,kpi_poetic,kpi_wp,kpi_noi score,kpi_wo score,kpi_shop score,kpi_make_ready score,kpi_rep score,kpi_npr score,kpi_vendor_compliance_score,kpi_dscr_12mo,Property Score Bucket
0,15,2025-11-01,Campus Court at Knollwood,Student,B,y,Student,Entrata Campus,2017-05-23,MI,Mapletree Investments,Scott Orphan,42.275247,'-85.621267,c1285p002352,Stacey Lecocke,current,Kyle Nail,Gabriela Arenas,Unassigned,<NA>,0.5,0.6,0.0,0.5,0.4,x,2.0,5826032.69,5820513.99,1699,2330,2348276.95,2516874.13,606471.93,638894.34,0.0,-9367.08,20898.19,22492204.49,<NA>,<NA>,4779376.79,5047282.59,488205.95999,678265.56999,5518.7,0.0009,-267905.8,-0.0531,y,-168597.18,-0.067,0.7292,0.9493,-0.4482,0.7198,1,<NA>,OK,OK,OK,<NA>,0.2,0.1,<NA>,<NA>,<NA>,0.0,0.125,0.25,0.125,1.0,0.5,0.25,2.00998,2_3


In [58]:
# Create connection
conn = duckdb.connect()
conn.register('df_view', df)


modified_query = """
SELECT *
FROM
    df_view
"""

modified_query = """
SELECT 
    *
FROM df_view
WHERE 
    prop_type_new = 'current'                                    -- Always required
    AND record_corporate_operating_office = 'Denver'            -- From office dropdown
    -- AND yoy_same_store = 'y'                                     -- If stabilized = true
    -- AND record_property_sub_type NOT LIKE '%New Development%'    -- If exclude_leaseup = true
"""

# modified_query = """
# SELECT 
#     record_corporate_operating_office, COUNT(DISTINCT record_look_up_code) as property_count
# FROM df_view
# group by 1
# """

# Execute your dynamic query
result = conn.execute(modified_query).df()
conn.close()

result

,record_record_id_,meas_mo,record_property_name,record_corporate_operating_office,record_asset_class,pms_linked,record_property_sub_type,record_accounting_system_of_record,record_takeover_date,record_state,record_ownership_group_name,record_svp_assignment,record_latitude,record_longitude,record_look_up_code,division_president,prop_type_new,record_vice_president_assignment,record_regional_assignment,record_regional_vp_assignment,record_rltd___rmd_assignment,Market Performance Score,Site Value & Risk Mgmt Score,Investment Results Score,Community Standards Score,Customer Experience,const,Property Score,kpi_gpr actual,kpi_gpr budget,kpi_completed wo within 48 hrs,kpi_total wo,kpi_noi actual,kpi_noi budget,kpi_collections,kpi_charges,kpi_delinquency,kpi_ap aging 60+ days,kpi_total ap,kpi_budget total,kpi_occupied,kpi_units,kpi_npr actual,kpi_npr budget,kpi_vend_pref_actl,kpi_vend_pref_elig,kpi_gpr variance,kpi_gpr variance %,kpi_npr variance,kpi_npr variance %,yoy_same_store,kpi_noi variance,kpi_noi variance %,kpi_wo completion %,kpi_collection %,kpi_ap_aging %,kpi_vendor_compliance %,kpi_risk_training_score,kpi_occupancy %,Exceptions Source,Exceptions Detail,Missing Budget,kpi_gpr score,kpi_ap aging score,kpi_collection score,kpi_lockton,kpi_poetic,kpi_wp,kpi_noi score,kpi_wo score,kpi_shop score,kpi_make_ready score,kpi_rep score,kpi_npr score,kpi_vendor_compliance_score,kpi_dscr_12mo,Property Score Bucket
0,2485,2025-11-01,Amberley at Inverness,Denver,A,y,Market Rate,Yardi - AssetLiving,2020-03-25,CO,Granite Capital Group,Unassigned,39.573101,'-104.867014,c1529p001008,Carol Nelson,current,Unassigned,Sherri Champignon,Chanele Richardson,None,0.0,0.35,0.0,0.38,0.1825,x,0.91,2873688.00,0.00,300,538,1538663.06,0.00,227312.0,223111.0,0.0,410.91,19154.30,5180810.72,81,96,2470191.70,0.00,240759.55999,1.055347e+06,2873688.00,NaN,2470191.70,NaN,y,1538663.06,NaN,0.5576,1.0188,0.0215,0.2281,1,0.8438,OK,OK,No NPR Budget No GPR Budget No NOI Budget,NaN,0.1,0.2,NaN,NaN,NaN,0.0,0.000,0.25,0.125,1.0,0.0,0.00,3.72745,0_1
1,2487,2025-11-01,ArtWalk at CityCenter,Denver,A,y,Market Rate,Yardi - AssetLiving,2020-10-20,CO,Oak Coast Properties,Unassigned,39.65515,'-104.99681,c1536p001023,Carol Nelson,current,Unassigned,Noah Alexander,Carla Schumacher,None,0.5,0.25,0.0,0.50,0.3125,x,1.56,7915840.00,8032383.44,1884,2381,5075374.96,5491385.94,701831.0,745070.0,0.0,3518.36,103884.89,28465403.26,407,437,7124982.88,7399048.51,527705.41999,1.141279e+06,-116543.44,-0.0145,-274065.63,-0.0370,y,-416010.98,-0.0758,0.7913,0.9420,0.0339,0.4624,1,0.9314,OK,OK,OK,NaN,0.1,0.1,NaN,NaN,NaN,0.0,0.125,0.25,0.125,1.0,0.5,0.00,2.61410,1_2
2,2490,2025-11-01,Alpine Flats,Denver,C,y,Market Rate,Yardi - AssetLiving,2021-07-09,CO,Oak Coast Properties,Unassigned,39.72823,'-105.051005,c1536p008381,Carol Nelson,current,Unassigned,Sherry Haney,Carla Schumacher,None,0.0,0.30,0.0,0.50,0.2000,x,1.00,3150139.00,3318317.75,619,917,644715.45,1638753.31,202782.0,245257.0,0.0,100892.53,182111.24,14301905.55,181,230,2111940.46,2905465.24,381468.25999,6.173894e+05,-168178.75,-0.0507,-793524.78,-0.2731,y,-994037.86,-0.6066,0.6750,0.8268,0.5540,0.6179,1,0.7870,OK,OK,OK,NaN,0.0,0.0,NaN,NaN,NaN,0.0,0.125,0.25,0.125,1.0,0.0,0.25,0.61151,1_2
3,2492,2025-11-01,Avilla Buffalo Run,Denver,A,n,Market Rate,Yardi - Nexmetro,2020-04-27,CO,NexMetro,Unassigned,39.91348,'-104.802247,c1530p003002,Carol Nelson,current,Unassigned,Kristy Larche,Kendra Halpern,None,0.0,0.05,0.0,0.38,0.1075,x,0.54,0.00,0.00,0,<NA>,0.00,0.00,0.0,0.0,0.0,NaN,NaN,0.00,<NA>,<NA>,0.00,0.00,0.00000,0.000000e+00,0.00,NaN,0.00,NaN,y,0.00,NaN,NaN,NaN,NaN,NaN,1,NaN,Not Linked,Not Linked,Not Linked,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,0.25,0.125,1.0,0.0,NaN,NaN,0_1
4,2493,2025-11-01,Avilla Eastlake,Denver,A,n,Market Rate,Yardi - Nexmetro,2020-11-30,CO,NexMetro,Unassigned,39.91959,'-104.973997,c1530p003001,Carol Nelson,current,Unassigned,Kristy Larche,Kendra Halpern,None,0.0,0.05,0.0,0.38,0.1075,x,0.54,0.00,0.00,0,<NA>,0.00,0.00,0.0,0.0,0.0,NaN,NaN

In [59]:
len(result['record_look_up_code'].unique())

104

In [17]:
result.to_csv('temp_agent_kpi_scorecard.csv', index=False)

## LLM Comparison

In [49]:
"""
Strategic Overview Report Generator
Sends property KPI data to OpenRouter (Gemini 2.5 Flash) to generate portfolio analysis
"""

import os
import requests
import pandas as pd
from typing import Optional, Tuple
from datetime import datetime


def prepare_dataframe_for_llm(df: pd.DataFrame, max_rows: int = 500) -> str:
    """
    Convert dataframe to a compact string format for LLM consumption.
    Uses CSV format as it's token-efficient.
    """
    if len(df) > max_rows:
        df = df.head(max_rows)
        print(f"Warning: Truncated to {max_rows} rows")
    
    return df.to_csv(index=False)


def build_system_prompt() -> str:
    """
    Build the system prompt for Strategic Overview generation.
    """
    return """You are a property management performance consultant specializing in portfolio analytics. 
Your role is to analyze property KPI data and deliver strategic insights for operational excellence.

REPORT TYPE: Strategic Overview
Focus on portfolio-wide insights, regional performance patterns, and strategic priorities.
Word Target: 500-800 words maximum.

KPI SCORING CONTEXT:
- Property Score ranges from 0.0-5.0 (5 categories worth 1.0 point each)
- The 5 category score columns are:
  * "Market Performance Score"
  * "Site Value & Risk Mgmt Score" 
  * "Investment Results Score"
  * "Community Standards Score"
  * "Customer Experience"
- Use natural scoring language: "Property scored 2.1" or "Community Standards is critically low at 0.3"

PERFORMANCE CLASSIFICATIONS:
- Critical: 0.0-1.5 (immediate intervention required)
- Low: 1.5-2.5 (significant underperformance)
- Moderate: 2.5-3.5 (below target)
- Good: 3.5-4.5 (solid performance)
- Excellent: 4.5-5.0 (outstanding)

PROPERTY COUNTING RULES:
- Count DISTINCT "record_look_up_code" values only (not property names)
- Use "record_property_name" for display purposes

DATA AUTHENTICITY REQUIREMENTS - CRITICAL:
- ONLY use specific numbers that come directly from the data provided
- When metrics aren't available, state "data not available" rather than estimating
- Never generate specific timeframes, percentages, or dollar amounts not in the data
- Averages must be calculated from actual property data provided
- Do NOT generate operational targets or benchmarks not present in the data

DATA COMPLETENESS INDICATORS:
- "Exceptions Source" column: "OK" means complete, other values indicate issues
- "Exceptions Detail" column: describes specific missing data
- "Missing Budget" column: identifies budget data gaps
- Properties with "Not Linked" in exceptions have incomplete PMS integration"""


def build_user_prompt(df_csv: str, office_filter: str, stabilized: bool = False, exclude_leaseup: bool = False) -> str:
    """
    Build the user prompt for Strategic Overview generation.
    """
    return f"""Analyze the following property KPI data and generate a Strategic Portfolio Overview report.

FILTERS APPLIED:
- Region: {office_filter}
- Stabilized Properties Only (yoy_same_store = 'y'): {stabilized}
- Exclude Lease-up/New Dev: {exclude_leaseup}

PROPERTY DATA (CSV format):
```
{df_csv}
```

Generate a STRATEGIC PORTFOLIO OVERVIEW with these exact sections:

---

PORTFOLIO HEALTH SCORECARD
- Overall portfolio average Property Score (calculate from data, format as X.XX/5.0)
- Total properties analyzed (count distinct record_look_up_code)
- Distribution by performance tier: count how many properties fall into Critical (0-1.5), Low (1.5-2.5), Moderate (2.5-3.5), Good (3.5-4.5), Excellent (4.5-5.0)
- Data completeness: count properties with "OK" in Exceptions Source vs those with issues

REGIONAL PERFORMANCE RANKING
Note: All properties are in the {office_filter} region, so analyze by sub-groupings available in the data:
- Performance by record_asset_class (A, B, C)
- Performance by record_property_sub_type (Market Rate, Affordable, etc.)
- Identify best/worst performing segments with actual scores

PORTFOLIO TRENDS & INSIGHTS
- Category-level analysis: Calculate average for each of the 5 category scores across the portfolio
  * Market Performance Score (avg)
  * Site Value & Risk Mgmt Score (avg)
  * Investment Results Score (avg)
  * Community Standards Score (avg)
  * Customer Experience (avg)
- Identify which categories are portfolio strengths vs weaknesses
- Note any systematic issues (e.g., many properties with low scores in same category)

PROPERTIES REQUIRING ATTENTION
- List up to 10-15 properties with Property Score < 2.5
- For each: property name, Property Score, and which category scores are dragging it down
- Prioritize by lowest scores first

STRATEGIC PRIORITIES
- Top 3-5 leadership priorities based on the data analysis
- Category-specific recommendations tied to actual portfolio weaknesses
- Which property segments need the most attention

REPORT METADATA
- Report Generated: [Current date]
- Analysis Period: Based on meas_mo column in data
- Properties Analyzed: [distinct count of record_look_up_code]
- Data Breakdown: [X complete / Y with data gaps]

---

FORMATTING REQUIREMENTS:
- Plain text only, NO HTML
- Use clear section headers as shown above
- Be concise but comprehensive (500-800 words)
- Every number must come from the actual data
- Do not invent benchmarks or targets"""


def build_strategic_overview_prompt(
    df_csv: str, 
    office_filter: str, 
    stabilized: bool = False, 
    exclude_leaseup: bool = False
) -> Tuple[str, str]:
    """
    Build the system and user prompts for Strategic Overview generation.
    
    Args:
        df_csv: Property data as CSV string
        office_filter: The region being analyzed
        stabilized: Whether stabilized filter was applied
        exclude_leaseup: Whether lease-up exclusion was applied
    
    Returns:
        Tuple of (system_prompt, user_prompt)
    """
    system_prompt = build_system_prompt()
    user_prompt = build_user_prompt(df_csv, office_filter, stabilized, exclude_leaseup)
    return system_prompt, user_prompt


def generate_strategic_overview(
    df: pd.DataFrame,
    office_filter: str,
    stabilized: bool = False,
    exclude_leaseup: bool = False,
    api_key: Optional[str] = None,
    model: str = "google/gemini-2.5-flash",
    temperature: float = 0.3,
    max_tokens: int = 4000
) -> str:
    """
    Send dataframe to OpenRouter and generate Strategic Overview report.
    
    Args:
        df: Filtered dataframe with property KPI data
        office_filter: The region being analyzed
        stabilized: Whether stabilized filter was applied
        exclude_leaseup: Whether lease-up exclusion was applied
        api_key: OpenRouter API key (or set OPENROUTER_API_KEY env var)
        model: Model to use (default: google/gemini-2.5-flash)
        temperature: LLM temperature (default: 0.3 for consistent output)
        max_tokens: Max response tokens (default: 4000)
    
    Returns:
        Generated report as string
    """
    # Get API key
    api_key = api_key or os.environ.get("OPENROUTER_API_KEY")
    if not api_key:
        raise ValueError("API key required. Pass api_key or set OPENROUTER_API_KEY env var")
    
    # Prepare data
    df_csv = prepare_dataframe_for_llm(df)
    system_prompt, user_prompt = build_strategic_overview_prompt(
        df_csv, office_filter, stabilized, exclude_leaseup
    )
    
    # Build request
    url = "https://openrouter.ai/api/v1/chat/completions"
    headers = {
        "Authorization": f"Bearer {api_key}",
        "Content-Type": "application/json",
        "HTTP-Referer": "https://asset-living.com",
        "X-Title": "KPI Strategic Overview Generator"
    }
    
    payload = {
        "model": model,
        "messages": [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ],
        "temperature": temperature,
        "max_tokens": max_tokens
    }
    
    # Log request info
    unique_properties = df['record_look_up_code'].nunique()
    print(f"Sending {len(df)} rows ({unique_properties} unique properties) to {model}...")
    
    # Make request
    response = requests.post(url, headers=headers, json=payload)
    response.raise_for_status()
    
    result = response.json()
    return result["choices"][0]["message"]["content"]


def print_report(report: str, title: str = "STRATEGIC PORTFOLIO OVERVIEW") -> None:
    """
    Pretty print the generated report.
    """
    print("\n" + "=" * 60)
    print(title)
    print("=" * 60 + "\n")
    print(report)


def save_report(report: str, filepath: str = "strategic_overview_output.txt") -> None:
    """
    Save the report to a text file.
    """
    with open(filepath, "w") as f:
        f.write(report)
    print(f"Report saved to {filepath}")

In [50]:
report = generate_strategic_overview(
    df=result,
    office_filter="Atlanta",
    stabilized=True,
    exclude_leaseup=False,
    api_key=OPEN_ROUTER_KEY,
    # model="google/gemini-3-pro-preview", # no response
    model="google/gemini-2.5-flash",
    # model="google/gemini-2.5-pro", # bad
    # model="anthropic/claude-sonnet-4.5"
    # model="openai/gpt-5-mini" # no response
)

Sending 209 rows (209 unique properties) to google/gemini-2.5-flash...


In [51]:
# Display it
print_report(report)


STRATEGIC PORTFOLIO OVERVIEW

PORTFOLIO HEALTH SCORECARD
- Overall portfolio average Property Score: 2.06/5.0
- Total properties analyzed: 101
- Distribution by performance tier:
  - Critical (0.0-1.5): 39 properties
  - Low (1.5-2.5): 23 properties
  - Moderate (2.5-3.5): 23 properties
  - Good (3.5-4.5): 16 properties
  - Excellent (4.5-5.0): 0 properties
- Data completeness: 96 properties with "OK" in Exceptions Source, 5 properties with data issues.

REGIONAL PERFORMANCE RANKING
- Performance by record_asset_class:
  - Class A: Average Property Score 1.89
  - Class B: Average Property Score 2.19
  - Class C: Average Property Score 1.76
  - The lowest performing asset class is Class C with an average score of 1.76.
- Performance by record_property_sub_type:
  - Affordable: Average Property Score 1.86
  - Affordable;New Development: Average Property Score 1.83
  - Affordable;Senior: Average Property Score 1.48
  - Affordable;Senior;New Development: Average Property Score 1.86
  - Af

In [ ]:


# Optionally save to file
save_report(report, "atlanta_strategic_overview.txt")

## Precompute metrics

In [ ]:
"""
Strategic Overview Report Generator
Pre-computes all statistics with pandas, uses LLM only for narrative generation.
"""

import os
import requests
import pandas as pd
from typing import Optional, Tuple, Dict, Any, List
from datetime import datetime


# ============================================================
# STATISTICS COMPUTATION (Pandas - Deterministic)
# ============================================================

def compute_portfolio_stats(df: pd.DataFrame) -> Dict[str, Any]:
    """
    Pre-compute all portfolio statistics using pandas.
    Returns a dictionary with all metrics needed for the report.
    """
    stats = {}
    
    # Basic counts
    stats['total_properties'] = df['record_look_up_code'].nunique()
    stats['total_rows'] = len(df)
    
    # Property Score statistics
    stats['avg_property_score'] = round(df['Property Score'].mean(), 2)
    stats['min_property_score'] = round(df['Property Score'].min(), 2)
    stats['max_property_score'] = round(df['Property Score'].max(), 2)
    
    # Score distribution by tier
    score_col = 'Property Score'
    stats['tier_distribution'] = {
        'critical': len(df[(df[score_col] >= 0) & (df[score_col] < 1.5)]),
        'low': len(df[(df[score_col] >= 1.5) & (df[score_col] < 2.5)]),
        'moderate': len(df[(df[score_col] >= 2.5) & (df[score_col] < 3.5)]),
        'good': len(df[(df[score_col] >= 3.5) & (df[score_col] < 4.5)]),
        'excellent': len(df[df[score_col] >= 4.5])
    }
    
    # Data completeness
    if 'Exceptions Source' in df.columns:
        # Count properties where Exceptions Source contains only "OK" (with possible whitespace)
        complete_mask = df['Exceptions Source'].astype(str).str.strip().str.upper() == 'OK'
        stats['data_complete'] = complete_mask.sum()
        stats['data_incomplete'] = len(df) - stats['data_complete']
    else:
        stats['data_complete'] = 'N/A'
        stats['data_incomplete'] = 'N/A'
    
    # Category score averages
    category_columns = [
        'Market Performance Score',
        'Site Value & Risk Mgmt Score',
        'Investment Results Score',
        'Community Standards Score',
        'Customer Experience'
    ]
    
    stats['category_averages'] = {}
    for col in category_columns:
        if col in df.columns:
            stats['category_averages'][col] = round(df[col].mean(), 2)
    
    # Performance by asset class
    if 'record_asset_class' in df.columns:
        stats['by_asset_class'] = (
            df.groupby('record_asset_class')['Property Score']
            .agg(['mean', 'count'])
            .round(2)
            .to_dict('index')
        )
    
    # Performance by property sub type
    if 'record_property_sub_type' in df.columns:
        stats['by_property_subtype'] = (
            df.groupby('record_property_sub_type')['Property Score']
            .agg(['mean', 'count'])
            .round(2)
            .to_dict('index')
        )
    
    # Properties requiring attention (score < 2.5), sorted by score ascending
    attention_df = df[df['Property Score'] < 2.5].sort_values('Property Score').head(15)
    stats['properties_requiring_attention'] = []
    
    for _, row in attention_df.iterrows():
        prop_info = {
            'name': row['record_property_name'],
            'score': round(row['Property Score'], 2),
            'category_scores': {}
        }
        for col in category_columns:
            if col in df.columns:
                prop_info['category_scores'][col] = round(row[col], 2)
        stats['properties_requiring_attention'].append(prop_info)
    
    # Metadata
    stats['report_generated'] = datetime.now().strftime('%Y-%m-%d %H:%M')
    if 'meas_mo' in df.columns:
        stats['analysis_period'] = df['meas_mo'].iloc[0]
    else:
        stats['analysis_period'] = 'N/A'
    
    return stats


def format_stats_for_llm(stats: Dict[str, Any], office_filter: str, stabilized: bool, exclude_leaseup: bool) -> str:
    """
    Format pre-computed statistics into a structured text summary for the LLM.
    """
    lines = []
    
    lines.append("=" * 60)
    lines.append("PRE-COMPUTED PORTFOLIO STATISTICS")
    lines.append("=" * 60)
    lines.append("")
    
    # Filters applied
    lines.append("FILTERS APPLIED:")
    lines.append(f"- Region: {office_filter}")
    lines.append(f"- Stabilized Properties Only: {stabilized}")
    lines.append(f"- Exclude Lease-up/New Dev: {exclude_leaseup}")
    lines.append("")
    
    # Portfolio overview
    lines.append("PORTFOLIO OVERVIEW:")
    lines.append(f"- Total Properties: {stats['total_properties']}")
    lines.append(f"- Average Property Score: {stats['avg_property_score']}/5.0")
    lines.append(f"- Score Range: {stats['min_property_score']} to {stats['max_property_score']}")
    lines.append("")
    
    # Score distribution
    lines.append("SCORE DISTRIBUTION BY TIER:")
    tier = stats['tier_distribution']
    lines.append(f"- Critical (0.0-1.5): {tier['critical']} properties")
    lines.append(f"- Low (1.5-2.5): {tier['low']} properties")
    lines.append(f"- Moderate (2.5-3.5): {tier['moderate']} properties")
    lines.append(f"- Good (3.5-4.5): {tier['good']} properties")
    lines.append(f"- Excellent (4.5-5.0): {tier['excellent']} properties")
    lines.append("")
    
    # Data completeness
    lines.append("DATA COMPLETENESS:")
    lines.append(f"- Complete (OK): {stats['data_complete']} properties")
    lines.append(f"- With Issues: {stats['data_incomplete']} properties")
    lines.append("")
    
    # Category averages
    lines.append("CATEGORY SCORE AVERAGES:")
    for cat, avg in stats['category_averages'].items():
        lines.append(f"- {cat}: {avg}")
    lines.append("")
    
    # By asset class
    if 'by_asset_class' in stats:
        lines.append("PERFORMANCE BY ASSET CLASS:")
        for asset_class, data in sorted(stats['by_asset_class'].items()):
            lines.append(f"- Class {asset_class}: Avg Score {data['mean']}, Count {int(data['count'])}")
        lines.append("")
    
    # By property subtype
    if 'by_property_subtype' in stats:
        lines.append("PERFORMANCE BY PROPERTY SUBTYPE:")
        # Sort by average score ascending to highlight worst performers
        sorted_subtypes = sorted(stats['by_property_subtype'].items(), key=lambda x: x[1]['mean'])
        for subtype, data in sorted_subtypes:
            lines.append(f"- {subtype}: Avg Score {data['mean']}, Count {int(data['count'])}")
        lines.append("")
    
    # Properties requiring attention
    lines.append("PROPERTIES REQUIRING ATTENTION (Score < 2.5, worst first):")
    for prop in stats['properties_requiring_attention']:
        lines.append(f"\n  {prop['name']} - Property Score: {prop['score']}")
        lines.append("  Category Breakdown:")
        for cat, score in prop['category_scores'].items():
            lines.append(f"    - {cat}: {score}")
    lines.append("")
    
    # Metadata
    lines.append("REPORT METADATA:")
    lines.append(f"- Report Generated: {stats['report_generated']}")
    lines.append(f"- Analysis Period: {stats['analysis_period']}")
    lines.append(f"- Properties Analyzed: {stats['total_properties']}")
    lines.append(f"- Data Breakdown: {stats['data_complete']} complete / {stats['data_incomplete']} with gaps")
    
    return "\n".join(lines)


# ============================================================
# PROMPT CONSTRUCTION
# ============================================================

def build_system_prompt() -> str:
    """
    Build the system prompt for Strategic Overview narrative generation.
    """
    return """You are a property management performance consultant specializing in portfolio analytics.
Your role is to write executive-level strategic reports based on pre-computed statistics.

IMPORTANT: All statistics have been pre-computed and verified. Your job is to:
1. Write clear, professional narrative around the provided numbers
2. Identify patterns and insights from the data
3. Provide strategic recommendations based on the findings
4. Use EXACTLY the numbers provided - do not recalculate or estimate

REPORT TYPE: Strategic Overview
Word Target: 500-800 words maximum.

PERFORMANCE CLASSIFICATIONS:
- Critical: 0.0-1.5 (immediate intervention required)
- Low: 1.5-2.5 (significant underperformance)
- Moderate: 2.5-3.5 (below target)
- Good: 3.5-4.5 (solid performance)
- Excellent: 4.5-5.0 (outstanding)

WRITING STYLE:
- Professional, executive-level tone
- Focus on insights and actionable recommendations
- Use the exact numbers provided in the statistics
- Highlight key patterns and concerns
- Be concise but comprehensive"""


def build_user_prompt(stats_summary: str) -> str:
    """
    Build the user prompt with pre-computed statistics.
    """
    return f"""Based on the following pre-computed portfolio statistics, write a Strategic Portfolio Overview report.

{stats_summary}

Write the report with these exact sections:

---

PORTFOLIO HEALTH SCORECARD
Summarize the overall portfolio health using the exact statistics provided. Include the average score, total properties, tier distribution, and data completeness.

REGIONAL PERFORMANCE RANKING
Analyze performance by asset class and property subtype. Identify the best and worst performing segments. Use the exact averages provided.

PORTFOLIO TRENDS & INSIGHTS
Discuss the category score averages. Identify which categories are strengths vs weaknesses. Note any systematic issues visible in the data.

PROPERTIES REQUIRING ATTENTION
Summarize the properties that need attention. Group them by common issues if patterns exist. Reference specific properties and their problem categories.

STRATEGIC PRIORITIES
Based on the data analysis, provide 3-5 specific leadership priorities. Tie recommendations to the actual weaknesses shown in the statistics.

REPORT METADATA
Include the exact metadata values provided.

---

FORMATTING REQUIREMENTS:
- Plain text only, NO HTML
- Use clear section headers as shown above
- Be concise but comprehensive (500-800 words)
- Use EXACTLY the numbers from the statistics - do not recalculate
- Focus on narrative, insights, and recommendations"""


def build_strategic_overview_prompt(
    stats: Dict[str, Any],
    office_filter: str,
    stabilized: bool = False,
    exclude_leaseup: bool = False
) -> Tuple[str, str]:
    """
    Build the system and user prompts for Strategic Overview generation.
    
    Args:
        stats: Pre-computed statistics dictionary
        office_filter: The region being analyzed
        stabilized: Whether stabilized filter was applied
        exclude_leaseup: Whether lease-up exclusion was applied
    
    Returns:
        Tuple of (system_prompt, user_prompt)
    """
    system_prompt = build_system_prompt()
    stats_summary = format_stats_for_llm(stats, office_filter, stabilized, exclude_leaseup)
    user_prompt = build_user_prompt(stats_summary)
    return system_prompt, user_prompt


# ============================================================
# REPORT GENERATION
# ============================================================

def generate_strategic_overview(
    df: pd.DataFrame,
    office_filter: str,
    stabilized: bool = False,
    exclude_leaseup: bool = False,
    api_key: Optional[str] = None,
    model: str = "google/gemini-2.5-flash",
    temperature: float = 0.3,
    max_tokens: int = 4000
) -> Tuple[str, Dict[str, Any]]:
    """
    Generate Strategic Overview report by pre-computing stats and using LLM for narrative.
    
    Args:
        df: Filtered dataframe with property KPI data
        office_filter: The region being analyzed
        stabilized: Whether stabilized filter was applied
        exclude_leaseup: Whether lease-up exclusion was applied
        api_key: OpenRouter API key (or set OPENROUTER_API_KEY env var)
        model: Model to use (default: google/gemini-2.5-flash)
        temperature: LLM temperature (default: 0.3 for consistent output)
        max_tokens: Max response tokens (default: 4000)
    
    Returns:
        Tuple of (generated report string, pre-computed stats dictionary)
    """
    # Get API key
    api_key = api_key or os.environ.get("OPENROUTER_API_KEY")
    if not api_key:
        raise ValueError("API key required. Pass api_key or set OPENROUTER_API_KEY env var")
    
    # Step 1: Pre-compute all statistics with pandas
    print(f"Computing statistics for {df['record_look_up_code'].nunique()} properties...")
    stats = compute_portfolio_stats(df)
    
    # Step 2: Build prompts with pre-computed stats
    system_prompt, user_prompt = build_strategic_overview_prompt(
        stats, office_filter, stabilized, exclude_leaseup
    )
    
    # Step 3: Call LLM for narrative generation only
    url = "https://openrouter.ai/api/v1/chat/completions"
    headers = {
        "Authorization": f"Bearer {api_key}",
        "Content-Type": "application/json",
        "HTTP-Referer": "https://asset-living.com",
        "X-Title": "KPI Strategic Overview Generator"
    }
    
    payload = {
        "model": model,
        "messages": [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ],
        "temperature": temperature,
        "max_tokens": max_tokens
    }
    
    print(f"Generating narrative with {model}...")
    response = requests.post(url, headers=headers, json=payload)
    response.raise_for_status()
    
    result = response.json()
    report = result["choices"][0]["message"]["content"]
    
    return report, stats


# ============================================================
# OUTPUT FUNCTIONS
# ============================================================

def print_report(report: str, title: str = "STRATEGIC PORTFOLIO OVERVIEW") -> None:
    """
    Pretty print the generated report.
    """
    print("\n" + "=" * 60)
    print(title)
    print("=" * 60 + "\n")
    print(report)


def print_stats(stats: Dict[str, Any]) -> None:
    """
    Print the pre-computed statistics for verification.
    """
    print("\n" + "=" * 60)
    print("PRE-COMPUTED STATISTICS (for verification)")
    print("=" * 60)
    print(f"Total Properties: {stats['total_properties']}")
    print(f"Avg Property Score: {stats['avg_property_score']}")
    print(f"Tier Distribution: {stats['tier_distribution']}")
    print(f"Data Complete: {stats['data_complete']}, Incomplete: {stats['data_incomplete']}")
    print(f"Category Averages: {stats['category_averages']}")


def save_report(report: str, filepath: str = "strategic_overview_output.txt") -> None:
    """
    Save the report to a text file.
    """
    with open(filepath, "w") as f:
        f.write(report)
    print(f"Report saved to {filepath}")


def save_report_to_pdf(
    report: str,
    filepath: str = "strategic_overview_output.pdf",
    title: str = "Strategic Portfolio Overview",
    office_filter: str = ""
) -> None:
    """
    Save the report to a PDF file using reportlab.
    
    Args:
        report: The generated report text
        filepath: Output PDF path
        title: Report title for the PDF header
        office_filter: Region name for subtitle
    """
    from reportlab.lib.pagesizes import letter
    from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle
    from reportlab.lib.units import inch
    from reportlab.lib.colors import HexColor
    from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer
    from reportlab.lib.enums import TA_CENTER
    
    # Create document
    doc = SimpleDocTemplate(
        filepath,
        pagesize=letter,
        rightMargin=0.75 * inch,
        leftMargin=0.75 * inch,
        topMargin=0.75 * inch,
        bottomMargin=0.75 * inch
    )
    
    # Get base styles and create custom ones
    styles = getSampleStyleSheet()
    
    # Custom styles
    styles.add(ParagraphStyle(
        name='ReportTitle',
        parent=styles['Title'],
        fontSize=18,
        spaceAfter=6,
        textColor=HexColor('#2c3e50')
    ))
    
    styles.add(ParagraphStyle(
        name='ReportSubtitle',
        parent=styles['Normal'],
        fontSize=12,
        spaceAfter=20,
        textColor=HexColor('#7f8c8d'),
        alignment=TA_CENTER
    ))
    
    styles.add(ParagraphStyle(
        name='SectionHeader',
        parent=styles['Heading2'],
        fontSize=12,
        spaceBefore=16,
        spaceAfter=8,
        textColor=HexColor('#2c3e50')
    ))
    
    styles.add(ParagraphStyle(
        name='ReportBodyText',
        parent=styles['Normal'],
        fontSize=10,
        spaceBefore=4,
        spaceAfter=4,
        leading=14
    ))
    
    styles.add(ParagraphStyle(
        name='ReportBulletText',
        parent=styles['Normal'],
        fontSize=10,
        leftIndent=20,
        spaceBefore=2,
        spaceAfter=2,
        leading=14
    ))
    
    # Build content
    story = []
    
    # Title
    story.append(Paragraph(title, styles['ReportTitle']))
    if office_filter:
        story.append(Paragraph(f"{office_filter} Region", styles['ReportSubtitle']))
    story.append(Spacer(1, 12))
    
    # Process report text into paragraphs
    lines = report.split('\n')
    
    for line in lines:
        line = line.strip()
        
        if not line:
            story.append(Spacer(1, 6))
            continue
        
        # Escape HTML special characters
        line = line.replace('&', '&amp;').replace('<', '&lt;').replace('>', '&gt;')
        
        # Detect section headers
        if line.isupper() or line.endswith('SCORECARD') or line.endswith('RANKING') or \
           line.endswith('INSIGHTS') or line.endswith('ATTENTION') or \
           line.endswith('PRIORITIES') or line.endswith('METADATA') or \
           line.startswith('PORTFOLIO') or line.startswith('REGIONAL') or \
           line.startswith('STRATEGIC') or line.startswith('PROPERTIES') or \
           line.startswith('REPORT'):
            story.append(Spacer(1, 8))
            story.append(Paragraph(f"<b>{line}</b>", styles['SectionHeader']))
        # Detect bullet points
        elif line.startswith('- ') or line.startswith('• ') or line.startswith('* '):
            bullet_text = line[2:].strip()
            story.append(Paragraph(f"• {bullet_text}", styles['ReportBulletText']))
        # Detect numbered items
        elif len(line) > 2 and line[0].isdigit() and line[1] in '.):':
            story.append(Paragraph(line, styles['ReportBulletText']))
        # Regular paragraph
        else:
            story.append(Paragraph(line, styles['ReportBodyText']))
    
    # Build PDF
    doc.build(story)
    print(f"Report saved to {filepath}")

In [60]:
# Generate the report (returns both report and stats)
report, stats = generate_strategic_overview(
    df=result,
    office_filter="Denver",
    stabilized=False,
    exclude_leaseup=False,
    api_key=OPEN_ROUTER_KEY  # Or set OPENROUTER_API_KEY env var
)

Computing statistics for 104 properties...
Generating narrative with google/gemini-2.5-flash...


In [61]:
# Verify the pre-computed statistics
print_stats(stats)


PRE-COMPUTED STATISTICS (for verification)
Total Properties: 104
Avg Property Score: 1.77
Tier Distribution: {'critical': 60, 'low': 12, 'moderate': 21, 'good': 11, 'excellent': 0}
Data Complete: 91, Incomplete: 13
Category Averages: {'Market Performance Score': 0.39, 'Site Value & Risk Mgmt Score': 0.31, 'Investment Results Score': 0.29, 'Community Standards Score': 0.43, 'Customer Experience': 0.35}


In [62]:
# Display the narrative report
print_report(report)


STRATEGIC PORTFOLIO OVERVIEW

STRATEGIC PORTFOLIO OVERVIEW: DENVER REGION

REPORT GENERATED: 2025-11-26 12:38
ANALYSIS PERIOD: 2025-11-01 00:00:00

PORTFOLIO HEALTH SCORECARD

The Denver portfolio, comprising 104 properties, exhibits a concerning overall performance with an average property score of 1.77 out of 5.0. This places the portfolio firmly in the "Low" performance classification. The score range, from 0.54 to 4.02, indicates significant disparity across assets, but the vast majority are underperforming. A critical finding is that 60 properties (57.7% of the portfolio) fall into the "Critical" tier (0.0-1.5), and an additional 12 properties are in the "Low" tier (1.5-2.5). This means 72 properties (69.2%) are performing below acceptable levels. Only 11 properties achieved a "Good" score (3.5-4.5), and notably, zero properties reached an "Excellent" classification (4.5-5.0). Data completeness is strong for 91 properties, with 13 properties identified as having data issues, whic

In [68]:
# Save to PDF
save_report_to_pdf(
    report,
    filepath="denver_strategic_overview.pdf",
    title="Strategic Portfolio Overview",
    office_filter="Denver"
)

KeyError: "Style 'BodyText' already defined in stylesheet"

# End Notebook